In [1]:
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
import os

Face Pattern Recognition

In [12]:
#Define image to work with. It represents the real classroom photo taken with the camera
classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos"
#Select the model to be used for detection varying the list index
detectors = [ "mtcnn","opencv", "retinaface"]
#Change depending on the index to be evaluates
chosen_detector_index=2

m=1
#Iterate through all real classroom photos
for file in os.listdir(classroomFolder):
    facesCoordinatesList=[]
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\{file}' 

    image = cv2.imread(camera_local_file)
    faces = DeepFace.extract_faces(img_path=camera_local_file, detector_backend=detectors[2])
    #Obtain images dimensions
    image_height, image_width, _ = image.shape

    #Save the coordinates of all faces in order to paint it later
    for i in range(len(faces)):
        facesCoordinatesList.append(faces[i]["facial_area"])
    # Draw bounding boxes for each detected face using the coordinates
    for element in facesCoordinatesList:
        left = element["x"]
        top = element["y"]
        width = element["w"]
        height =  element["h"]
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (0,0,200), 
            2  
        )

    # Print number of faces detected
    print(f"{len(facesCoordinatesList)} Faces were detected in {camera_local_file}")

    # Save the annotated image in the destinated folder
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Detection\Detection{detectors[chosen_detector_index]}\faces_detected_{m}.jpeg"
    
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    m=m+1


20 Faces were detected in C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\classroom1.jpeg
14 Faces were detected in C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\classroom2.jpeg


Face Recognition

In [45]:
#student_names_array=["AlejandroCallasas", "AndresAnillo","DiegoGomez","GabrielCastanez", "MauricioDeLaHoz","YordiRochel", "MariaBerdugo","AlbaQuiroz","GiohanOlivares","JesusCotes"]
#colors=[(0, 200, 0),(200,0,0),(0,0,0), (0,0,255), (150,150,0),(0,150,150),(100,20,60),(20,60,100),(60,100,20),(255,255,255)]
student_names_array=["YordiRochel", "MariaBerdugo","AlbaQuiroz","GiohanOlivares","JesusCotes"]
colors=[(0, 200, 0),(200,0,0),(0,0,0), (0,0,255), (150,150,0)]


recognition_models=["ArcFace","Facenet", "VGG-Face"]
chosen_model_index=1

classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos"
model=recognition_models[chosen_model_index]

boundingBoxRecognizedFaces=[]
#os.makedirs(output_folder, exist_ok=True)
m=1
for file in os.listdir(classroomFolder):
    camera_local_file=rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\{file}"
    #The image read of the classroom. This will be used to redraw it
    camera_photo = cv2.imread(camera_local_file)
    recognized_counter=0
    k=0
    boundingBoxRecognizedFacesithImage=[]

    for student_name in student_names_array:
        student_photo_path=rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Students\{student_name}.jpeg"

        result = DeepFace.verify(
                    img1_path=student_photo_path, #individual
                    img2_path=camera_local_file,
                    model_name=model,
                    detector_backend="mtcnn",
                    enforce_detection=False
                    )
        coordinatesOfGroupPhoto=result["facial_areas"]["img2"]
        x=coordinatesOfGroupPhoto["x"]
        y=coordinatesOfGroupPhoto["y"]
        h=coordinatesOfGroupPhoto["h"]
        w=coordinatesOfGroupPhoto["w"]


            # Cargar la imagen
        if camera_photo is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {classroomFolder}")

        # Dibujar el rectángulo
        if str(result["verified"])=="True":
            x, y, w, h = coordinatesOfGroupPhoto["x"], coordinatesOfGroupPhoto["y"], coordinatesOfGroupPhoto["w"], coordinatesOfGroupPhoto["h"]
            cv2.rectangle(camera_photo, (x, y), (x + w, y + h), colors[k], 2)  # Color verde, grosor 2
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(camera_photo, f"{student_name}", (x + 10, y - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)
            print(f"{student_name} was recognized in photo {m}")
            recognized_counter=recognized_counter+1

            boundingBoxRecognizedFacesithImage.append((student_name,x,y,h,w))

        else:
            print(f"{student_name} was NOT recognized in photo {m}")
            boundingBoxRecognizedFacesithImage.append((student_name,0,0,0,0))
        k=k+1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)


    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Recognition\Recognition{model}\faces_in_classroom{m}_{recognized_counter}.jpg"
        # Guardar la imagen modificada
    cv2.imwrite(output_path, camera_photo)
    print(f"Imagen guardada en: {output_path}")
    m=m+1

YordiRochel was NOT recognized in photo 1
MariaBerdugo was NOT recognized in photo 1
AlbaQuiroz was NOT recognized in photo 1
GiohanOlivares was recognized in photo 1
JesusCotes was NOT recognized in photo 1
Imagen guardada en: C:\Users\User\Desktop\FinalProject\ImagesDeepface\Recognition\RecognitionFacenet\faces_in_classroom1_1.jpg
YordiRochel was NOT recognized in photo 2
MariaBerdugo was NOT recognized in photo 2
AlbaQuiroz was NOT recognized in photo 2
GiohanOlivares was NOT recognized in photo 2
JesusCotes was NOT recognized in photo 2
Imagen guardada en: C:\Users\User\Desktop\FinalProject\ImagesDeepface\Recognition\RecognitionFacenet\faces_in_classroom2_0.jpg


Calculate IoU

In [ ]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou


[('YordiRochel', 0, 0, 0, 0),
 ('MariaBerdugo', 0, 0, 0, 0),
 ('AlbaQuiroz', 0, 0, 0, 0),
 ('GiohanOlivares', 0, 0, 0, 0),
 ('JesusCotes', 0, 0, 0, 0)]

Define tuples from baseline (Manual correction and verificatio)

In [ ]:
true=True
faceGallery=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":733,"y":392,"width":31,"height":44},"region_attributes":{"name":"AlejandroCallasas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":153,"y":475,"width":45,"height":46},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":275,"width":20,"height":27},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":13,"y":506,"width":37,"height":39},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":716,"y":348,"width":22,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":367,"y":421,"width":33,"height":39},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":362,"width":31,"height":35},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":908,"y":346,"width":27,"height":31},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1188,"y":326,"width":32,"height":36},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":114,"y":534,"width":44,"height":59},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}},
{"classroom2.jpeg148639":{"filename":"classroom2.jpeg","size":148639,"regions":[{"shape_attributes":{"name":"rect","x":147,"y":465,"width":56,"height":60},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":2,"y":496,"width":51,"height":64},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1206,"y":314,"width":46,"height":61},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":154,"y":551,"width":55,"height":74},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":347,"width":44,"height":60},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":408,"y":415,"width":44,"height":59},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


[[('AlejandroCallasas', 733, 392, 31, 44),
  ('AndresAnillo', 153, 475, 45, 46),
  ('DiegoGomez', 660, 275, 20, 27),
  ('GabrielCastanez', 13, 506, 37, 39),
  ('MauricioDeLaHoz', 716, 348, 22, 30),
  ('YordiRochel', 367, 421, 33, 39),
  ('MariaBerdugo', 1155, 362, 31, 35),
  ('AlbaQuiroz', 908, 346, 27, 31),
  ('GiohanOlivares', 1188, 326, 32, 36),
  ('JesusCotes', 114, 534, 44, 59)],
 [('AndresAnillo', 147, 465, 56, 60),
  ('GabrielCastanez', 2, 496, 51, 64),
  ('GiohanOlivares', 1206, 314, 46, 61),
  ('GiohanOlivares', 154, 551, 55, 74),
  ('MariaBerdugo', 1158, 347, 44, 60),
  ('YordiRochel', 408, 415, 44, 59)]]

Iterate through the base line and compare it to the results obtained by Deepface's models. Then calculate TP, TN, FP, FN

In [50]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]

    listOfNamesBaseline=[]
    for element in photoObject:
        listOfNamesBaseline.append(element[0])

    listOfNamesModel=[]
    for element in tuplesToCompare:
        listOfNamesModel.append(element[0])


    for element in photoObject:
        if(element[0] in listOfNamesModel):
            for identity in tuplesToCompare:
                print("-------------")
                IoU=calculate_iou(element,identity)
                #print(f"{element} compared with {identity}")

                #Case True Positive: Identity and bounding box match
                if(IoU>=0.5 and element[0]==identity[0]):
                    TP=TP+1

                #Case False Positive: no identity match, however, the bounding boxes are similar
                if(IoU>=0.5 and element[0]!=identity[0]):
                    FP=FP+1

                #Case True Negative: No bounding box match. However, the identity is different
                if(IoU<0.5 and element[0]!=identity[0]):
                    TN=TN+1

                #Case False Negative: The identity matches. However, bonding boxes do not correspond
                if(IoU<0.5 and element[0]==identity[0]):
                    FN=FN+1 
        else:
            FN=FN+1
            TN=TN+len(listOfNamesBaseline)-1
            print(f"{element[0]} not in model's list and adds 1 FN and {len(listOfNamesBaseline)-1} TN")


    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1


Results for photo 1
AlejandroCallasas not in model's list and adds 1 FN and 9 TN
AndresAnillo not in model's list and adds 1 FN and 9 TN
DiegoGomez not in model's list and adds 1 FN and 9 TN
GabrielCastanez not in model's list and adds 1 FN and 9 TN
MauricioDeLaHoz not in model's list and adds 1 FN and 9 TN
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
TP : 1 
TN : 65 
FP : 0 
FN : 9
Results for photo 2
AndresAnillo not in model's list and adds 1 FN and 5 TN
GabrielCastanez not in model's list and adds 1 FN and 5 TN
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
------

In [53]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]
    #Look for every bounding box drawn in the rekognition array. Element corresponds to the drawn rectangle, identify to rekognition's precdiction
    for element in photoObject:
        for identity in tuplesToCompare:
            IoU=calculate_iou(element,identity)
            #Case True Positive: Identity and bounding box match
            #print(f"{element} compared with {identity}")
            if(IoU>=0.5 and element[0]==identity[0]):
                TP=TP+1
            #Case False Positive: no identity match, however, the bounding boxes are similar
            if(IoU>=0.5 and element[0]!=identity[0]):
                FP=TP+1

            #Case True Negative: No bounding box match. However, the identity is different
            if(IoU<0.5 and element[0]!=identity[0]):
                TN=TN+1

            #Case False Negative: The identity matches. However, bonding boxes do not correspond
            if(IoU<0.5 and element[0]==identity[0]):
                FN=FN+1 


    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1


Results for photo 1
TP : 1 
TN : 45 
FP : 0 
FN : 4
Results for photo 2
TP : 0 
TN : 26 
FP : 0 
FN : 4


False

Mathematical metrics for analizing facial similarity

In [4]:
from scipy.spatial.distance import euclidean,cosine

#Obtain the analysis of the image
embedding1=DeepFace.represent(img_path="single2.jpeg",model_name="ArcFace", detector_backend="retinaface")
embedding2=DeepFace.represent(img_path="single11.jpeg",model_name="ArcFace", detector_backend="retinaface")

#Extract the multi-dimensional vector only
embedding1Vector1=np.array(embedding1[0]["embedding"])
embedding1Vector2=embedding2[0]["embedding"]

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

print("Cosine similarity is "+ str(cosine_similarity(embedding1Vector1,embedding1Vector2)))

def euclidean_distance(embedding1,embedding2):
    return euclidean(embedding1,embedding2)

print("Euclidean distance is "+ str(euclidean_distance(embedding1Vector1,embedding1Vector2)))



Cosine similarity is 0.35477153678865364
Euclidean distance is 3.8419692767006404
